In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib.backends.backend_pdf import PdfPages
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
#pd.options.mode.copy_on_write = True
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
import multiprocessing as mp
from tqdm import tqdm
from astropy.table import join
cosmo = FlatLambdaCDM(name='Planck18', H0=67.66, Om0=0.30966, Tcmb0=2.7255, Neff=3.046, m_nu=[0.  , 0.  , 0.06]* u.eV, Ob0=0.04897)
cores = 8

In [2]:
sys.path.append('../UploadToFritz')
import GetLightcurve

def get_fphot(data):
    objid, crossmatch = data
    df_photometry = GetLightcurve.full_dataframe([objid, crossmatch])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]
    
    df_photometry.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"], inplace = True)
    
    return df_photometry

In [3]:
interesting_df =  pd.read_csv('../MatchToDESI/interesting_desi.csv', index_col = "Unnamed: 0")
interesting_df.head()

,STANDARD_NAME,21A,22A,22B,23A,23B,RA,DEC,Z,Z_ERR,SPECTYPE,OBSERVATION_STATUS
0,A202204211423123m005116,NaN,A202204211423123m005116,NaN,NaN,NaN,215.801045,-0.854565,0.190317,0.000025,QSO,Y
1,T202210060350237m154301,NaN,NaN,T202210060350237m154301,NaN,NaN,57.598765,-15.716956,0.230973,0.000004,GALAXY,Y
2,C202210060349568m152123,NaN,NaN,C202210060349568m152123,NaN,NaN,57.486793,-15.356516,NaN,NaN,NaN,N
3,C202311050349356m152913,NaN,NaN,NaN,NaN,C202311050349356m152913,57.398318,-15.486978,0.143206,0.000002,GALAXY,Y
4,C202310060349100m161333,NaN,NaN,NaN,NaN,C202310060349100m161333,57.291634,-16.225747,NaN,NaN,NaN,N


In [4]:
queue = [[i, interesting_df] for i in interesting_df["STANDARD_NAME"]]
pool = mp.Pool(processes=cores)
interesting_lightcurve_dfs = list(tqdm(pool.imap_unordered(get_fphot, queue), total = len(queue)))
interesting_lightcurves_df = pd.concat(interesting_lightcurve_dfs)

100%|██████████| 649/649 [00:16<00:00, 38.62it/s]


In [5]:
interesting_lightcurves_table = Table.from_pandas(interesting_lightcurves_df)
interesting_table = Table.from_pandas(interesting_df)

In [9]:
salt_objects = Table.from_pandas(pd.read_csv("salt_objects.txt"))

In [10]:
merged_table = join(salt_objects, interesting_table, keys='STANDARD_NAME')

In [18]:
merged_table["STANDARD_NAME"]

A202104061415068p022542
A202104151408147m001144
C202104061410153p022017
C202212131051029m061317
C202212191049483m063944
C202212221055530m052824
T202204241509329p082207
T202301031055024m074329


In [17]:
coords = SkyCoord(merged_table["RA"], merged_table["DEC"], frame="icrs", unit="deg")
coords_string = coords.to_string('hmsdms')
coords_string

['14h15m06.8069s +02d25m42.3135s',
 '14h08m14.7051s -00d11m43.6752s',
 '14h10m15.338s +02d20m16.9281s',
 '10h51m02.8817s -06d13m17.4696s',
 '10h49m48.2887s -06d39m44.1794s',
 '10h55m52.9567s -05d28m24.4529s',
 '15h09m32.912s +08d22m06.6738s',
 '10h55m02.4446s -07d43m28.754s']

In [30]:
#https://fritz.science/source/C202212221055530m052824 still pretty bright 300 days ago
#https://fritz.science/source/A202104151408147m001144 Abs M is -23 according to Photo-z
#https://fritz.science/source/C202212131051029m061317 still pretty bright 300 days ago
#https://fritz.science/source/A202212191112192m054945 very slow fall